In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import cobra 
from cobra import Model, Reaction, Metabolite 
import pickle
from tqdm import tqdm 
from ast import literal_eval
import matplotlib.pyplot as plt
import sys
import pandas as pd
from rdkit import Chem
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.DataStructs import FingerprintSimilarity
from rdkit.Chem import AllChem
from tqdm import tqdm
from Bio import SeqIO
import pandas as pd
import pickle
from collections import Counter

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

### input and output

In [3]:
###input
yeast870_path =  '../../Data/model/yeast-GEM.yml'
# yeast8U_path = '../Data/model/yeast8U_plus_mod.yml'
yeast8U_path = '../../Data/model/yeast8U_lipid.yml'


# rxndb_to_model_total_info_path = '../Data/rxndb_total_info_to_model.csv'
rxndb_to_model_total_info_path = '../../Results/lipid/rxndb_total_info_to_model_yeast8_lipid.csv'


###output
# ss_ssn_snsn_reaction_ratio_figure_path = '../Figure_plus/fig2-c.pdf'
link_met_frequency_ipath3_file = '../../Results/iPath3/yeast8U_link_metabolite_ipath3_lipids.csv'


# get_underg_reaction_met_keggid_and_frequency

In [4]:
def get_underground_reaction_met_id(yeast8U_path):
    yeast8U = cobra.io.load_yaml_model(yeast8U_path)
    link_met = []
    for reac in yeast8U.reactions:
        if 'rxnl' in reac.id:######################################################################### lipids rxnl
            # if 's_' in reac.reaction and 'sn_' in reac.reaction:
            #    print([key.id for key,value in reac.metabolites.items() if 's_' in key.id])
                link_met += [key.id for key,value in reac.metabolites.items() if 's_' in key.id]
    link_met = set(link_met)
    print('link_met',len(link_met))    
    return link_met

In [5]:
link_met_id_lst = get_underground_reaction_met_id(yeast8U_path)
link_met_id_lst

link_met 261


{'s_0009',
 's_0010',
 's_0021',
 's_0025',
 's_0030',
 's_0042',
 's_0045',
 's_0048',
 's_0051',
 's_0054',
 's_0056',
 's_0066',
 's_0079',
 's_0126',
 's_0149',
 's_0154',
 's_0169',
 's_0176',
 's_0180',
 's_0185',
 's_0209',
 's_0211',
 's_0229',
 's_0231',
 's_0232',
 's_0234',
 's_0239',
 's_0243',
 's_0247',
 's_0250',
 's_0253',
 's_0257',
 's_0258',
 's_0260',
 's_0291',
 's_0296',
 's_0297',
 's_0322',
 's_0323',
 's_0335',
 's_0359',
 's_0362',
 's_0367',
 's_0373',
 's_0394',
 's_0409',
 's_0410',
 's_0411',
 's_0419',
 's_0423',
 's_0434',
 's_0442',
 's_0456',
 's_0467',
 's_0507',
 's_0512',
 's_0522',
 's_0526',
 's_0529',
 's_0558',
 's_0574',
 's_0580',
 's_0595',
 's_0605',
 's_0629',
 's_0633',
 's_0664',
 's_0666',
 's_0680',
 's_0683',
 's_0687',
 's_0689',
 's_0722',
 's_0739',
 's_0743',
 's_0745',
 's_0765',
 's_0767',
 's_0771',
 's_0794',
 's_0803',
 's_0816',
 's_0823',
 's_0837',
 's_0845',
 's_0900',
 's_0903',
 's_0929',
 's_0932',
 's_0952',
 's_0958',

In [6]:
def get_underg_reaction_met_keggid_and_frequency(yeast870_path,yeast8U_path,link_met_id_lst,link_met_frequency_ipath3_file):
    yeast8 = cobra.io.load_yaml_model(yeast870_path)
    yeast8U = cobra.io.load_yaml_model(yeast8U_path)
    link_metabolite_list = {'KEGG_id':[],
                    'color':[],
                    'frequency':[],
                    'width':[]}
    for i in link_met_id_lst:
        if 'kegg.compound' in yeast8.metabolites.get_by_id(i).annotation:
            link_metabolite_list['KEGG_id'].append(yeast8.metabolites.get_by_id(i).annotation['kegg.compound'])
            link_metabolite_list['color'].append('#d73027')
            link_metabolite_list['width'].append('W10') 
            frequency = 0
            for rea in yeast8U.reactions:
                if 'rxn' in rea.id:
                    if i in rea.reaction:
                        frequency +=1
            link_metabolite_list['frequency'].append(frequency) 
        else:pass
            # print('no kegg id',i)    
    link_met_keggid_and_frequency = pd.DataFrame(link_metabolite_list)
    # link_met_keggid_and_frequency.to_csv(link_met_frequency_ipath3_file,header=None,index=None,sep='\t')
    return link_met_keggid_and_frequency

In [7]:
underg_reaction_met_keggid_and_frequency = get_underg_reaction_met_keggid_and_frequency(yeast870_path,yeast8U_path,link_met_id_lst,link_met_frequency_ipath3_file)

In [8]:
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00001']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00080']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00009']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00002']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00008']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00003']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00004']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00006']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00005']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00011']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00020']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00007']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00013']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C01342']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00010']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00704']

underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00446']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00275']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C00085']
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency[underg_reaction_met_keggid_and_frequency['KEGG_id']!= 'C01094']

In [9]:
# get frequency list
frequency_list = underg_reaction_met_keggid_and_frequency['frequency'].to_list()
# sort the list
frequency_list.sort()
frequency_list

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 10,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 17,
 17,
 19,
 19,
 23,
 24,
 24,
 41,
 98,
 99,
 100,
 102,
 106,
 106,
 108,
 111,
 113,
 115,
 117,
 119,
 120,
 121,
 124,
 127,
 129,
 134,
 245,
 286,
 298,
 372,
 373,
 379,
 1044,
 1044,
 1168,
 1168]

In [10]:
underg_reaction_met_keggid_and_frequency['width'] = underg_reaction_met_keggid_and_frequency['frequency'].apply(lambda x: 'W' + str(x//100 * 5 + 10))
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W15','color'] = '#a8ddb5'#feb24c#fc9272
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W20','color'] = '#7bccc4'#fd8d3c#fb6a4a
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W25','color'] = '#4eb3d3'#fc4e2a#ef3b2c
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W30','color'] = '#2b8cbe'#e31a1c#cb181d
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W35','color'] = '#0868ac'#bd0026#a50f15
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W40','color'] = '#084081'#800026#67000d

underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W45','color'] = '#084081'
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W45','width'] = 'W40'

underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W50','color'] = '#084081'
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W50','width'] = 'W40'

underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W55','color'] = '#084081'
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W55','width'] = 'W40'

underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W60','color'] = '#084081'
underg_reaction_met_keggid_and_frequency.loc[underg_reaction_met_keggid_and_frequency['width']=='W60','width'] = 'W40'

underg_reaction_met_keggid_and_frequency[['KEGG_id','color','width']].to_csv(link_met_frequency_ipath3_file,header=None,index=None,sep='\t')
underg_reaction_met_keggid_and_frequency = underg_reaction_met_keggid_and_frequency.sort_values(by='frequency', ascending=False).reset_index(drop=True)
# underg_reaction_met_keggid_and_frequency[['KEGG_id','color','width','frequency']].to_csv('../Data/iPath3/yeast8_link_metabolite_frequency.csv',header=None,index=None,sep='\t')
underg_reaction_met_keggid_and_frequency

,KEGG_id,color,frequency,width
0,C00073,#d73027,1168,W65
1,C03172,#d73027,1168,W65
2,C00021,#084081,1044,W40
3,C00019,#084081,1044,W40
4,C00093,#4eb3d3,379,W25
...,...,...,...,...
168,C05264,#d73027,1,W10
169,C00197,#d73027,1,W10
170,C04411,#d73027,1,W10
171,C01835,#d73027,1,W10


In [11]:
underg_reaction_met_keggid_and_frequency.head(12)

,KEGG_id,color,frequency,width
0,C00073,#d73027,1168,W65
1,C03172,#d73027,1168,W65
2,C00021,#084081,1044,W40
3,C00019,#084081,1044,W40
4,C00093,#4eb3d3,379,W25
5,C00022,#4eb3d3,373,W25
6,C00074,#4eb3d3,372,W25
7,C00189,#7bccc4,298,W20
8,C00065,#7bccc4,286,W20
9,C00112,#7bccc4,245,W20


https://www.frontiersin.org/articles/10.3389/fcimb.2023.1240743/full
4.3 Methionine and phospholipid synthesis are related.